In [36]:
import pyarrow.parquet as pq
import pandas as pd
import re
from IPython.display import display

#dataset_1 = pq.ParquetDataset('dataset-1/w32processes')
#dataset_2 = pq.ParquetDataset('dataset-2/w32processes')
#dataset_3 = pq.ParquetDataset('dataset-3/w32processes')
dataset_4 = pq.ParquetDataset('../dataset-4/w32processes')

#table_1 = dataset_1.read()
#table_2 = dataset_2.read()
#table_3 = dataset_3.read()
table_4 = dataset_4.read()

#df_1 = table_1.to_pandas()
#df_2 = table_2.to_pandas()
#df_3 = table_3.to_pandas()
df = table_4.to_pandas()

#df_1.to_csv('test_1.csv')
#df_2.to_csv('test_2.csv')
#df_3.to_csv('test_3.csv')
#df.to_csv('test_4.csv')

pd.set_option('display.max_colwidth', None)

print('Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3')

Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3


In [37]:
ACCESSIBILITY_TARGETS = [
    "sethc.exe", "utilman.exe", "osk.exe", "magnify.exe",
    "narrator.exe", "atbroker.exe", "displayswitch.exe"
]

EXTRA_LOGON_RELATED = [
    "easeofaccessdialog.exe", "tabtip.exe", "ctfmon.exe"
]

SHELLS = ["cmd.exe", "powershell.exe", "pwsh.exe", "wscript.exe", "cscript.exe", "mshta.exe"]

SERVICE_LIKE = [
    "services.exe", "svchost.exe", "lsass.exe", "wininit.exe",
    "taskhostw.exe", "spoolsv.exe", "smss.exe", "csrss.exe",
    "dllhost.exe", "dcomlaunch.exe", "winlogon.exe", "wmiprvse.exe"
]


In [38]:
df_elevated = df[df['username'].str.contains('NT AUTHORITY')]

In [41]:
ACCESSIBILITY_TARGETS = [
    "sethc.exe", "utilman.exe", "osk.exe", "magnify.exe",
    "narrator.exe", "atbroker.exe", "displayswitch.exe"
]

def accessibility_tools(df):
    mask = df['name'].str.lower().apply(
        lambda x: any(t in x for t in ACCESSIBILITY_TARGETS)
    )
    return df[mask]
    
df1 = accessibility_tools(df)

display(df1)

print("There are no vulnerability tool executions?")

,hostname,name,arguments,username,path,pid


There are no vulnerability tool executions?


In [ ]:
def hunt2_accessibility_nonstd_path(df):
    mask_name = df['name'].str.lower().isin(ACCESSIBILITY_TARGETS)
    mask_nonstd = ~df['path'].str.lower().str.contains(r'\\windows\\(system32|syswow64)\\', regex=True, na=False)
    return df[mask_name & mask_nonstd]

In [ ]:
def hunt3_shells_referencing_accessibility(df):
    """Shells/scripts elevados cuyos argumentos mencionan accesibilidad"""
    mask_shells = df['name'].str.lower().isin(SHELLS)
    mask_args = df['arguments'].str.lower().str.contains('|'.join(ACCESSIBILITY_TARGETS), na=False)
    mask_elev = df['username'].apply(is_elevated)
    return df[mask_shells & mask_args & mask_elev]

In [ ]:
def hunt4_access_spawning_shell(df):
    """Accesibilidad ejecutando shells o comandos sospechosos"""
    suspicious_tokens = [
        " cmd", " powershell", " pwsh", " /c ",
        " -enc ", " -encodedcommand ", " -nop ", " -w hidden "
    ]
    mask_name = df['name'].str.lower().isin(ACCESSIBILITY_TARGETS + EXTRA_LOGON_RELATED)
    mask_args = df['arguments'].str.lower().str.contains('|'.join([re.escape(x) for x in suspicious_tokens]), na=False)
    return df[mask_name & mask_args]

In [ ]:
def hunt5_rare_elevated_processes(df, threshold=5):
    """Procesos elevados relacionados con accesibilidad/shells en menos de X hosts"""
    mask_elev = df['username'].apply(is_elevated)
    poi_mask = (
        df['name'].str.lower().isin(ACCESSIBILITY_TARGETS + EXTRA_LOGON_RELATED + SHELLS) |
        df['arguments'].str.lower().str.contains('|'.join(ACCESSIBILITY_TARGETS), na=False)
    )
    poi = df[mask_elev & poi_mask]
    host_counts = poi.groupby('name')['hostname'].nunique()
    rare_names = host_counts[host_counts < threshold].index
    return poi[poi['name'].isin(rare_names)]

In [ ]:
def hunt6_rare_services_elevated(df, threshold=5):
    """Servicios elevados sospechosos que existen en < threshold hosts"""
    mask_service_like = (
        df['name'].str.lower().isin(SERVICE_LIKE)
        | df['path'].str.lower().str.contains(r'\\(system32|syswow64)\\.*\\.exe', na=False, regex=True)
        | df['name'].str.lower().str.contains(r'(svc|service|host|win|sys)')
    )
    svc = df[mask_service_like & df['username'].apply(is_elevated)]
    host_counts = svc.groupby('name')['hostname'].nunique()
    rare = host_counts[host_counts < threshold].index
    return svc[svc['name'].isin(rare)]